In [52]:
import pandas as pd
import numpy as np
#from pymc_marketing.mmm.transformers import geometric_adstock, logistic_saturation, MMM
from sklearn.preprocessing import MaxAbsScaler
import pymc_marketing


In [20]:
mmm_data = pd.read_csv('media_mix_model_final_data_daily.csv')

In [21]:
mmm_data.columns

Index(['date_report', 'sp500', 'non_restricted_members', 'affiliates',
       'apple_search', 'google_search', 'bing_search', 'tik_tok', 'spotify',
       'pinterest', 'buzzfeed', 'influencer', 'ooh', 'audience_network',
       'facebook', 'instagram', 'total_paid_spend', 'elle_raise',
       'money_cheat_sheet', 'opinionated', 'weekly_events', 'tofu_emails',
       'instagram_impressions', 'campaigns', 'promos', 'campaigns_promos',
       'interview', 'other_appearance', 'podcast', 'speaking', 'tv',
       'sally_opinionated', 'non_restricted_tp', 'google_org', 'meta_app',
       'meta_web', 'audience_network_app', 'audience_network_web',
       'facebook_app', 'facebook_web', 'instagram_app', 'instagram_web',
       'unrestricted_new_member_executive', 'unrestricted_new_member_plus',
       'unrestricted_new_member_essential', 'pwm_leads_self_reporting',
       'pwm_leads_website', 'pwm_leads'],
      dtype='object')

Combine variables to simplify model and understand impact of overall marketing channel

In [22]:
mmm_data['paid_meta'] = mmm_data['instagram'] +  mmm_data['facebook'] + mmm_data['audience_network']
mmm_data['newletters'] = mmm_data['money_cheat_sheet'] +  mmm_data['opinionated'] + mmm_data['weekly_events']

Select variables that had impact in original model

In [23]:
mmm_data = mmm_data[['date_report', 'non_restricted_members', 'instagram_impressions', 'paid_meta', 'sp500', 'google_search', 'google_org', 'newletters', 'tofu_emails', 'affiliates' ]]

Rename variables for clarity 

In [24]:
mmm_data = mmm_data.rename(columns={'non_restricted_members': 'new_clients',
                                    'instagram_impressions': 'instagram_org'})

In [25]:
mmm_data.columns

Index(['date_report', 'new_clients', 'instagram_org', 'paid_meta', 'sp500',
       'google_search', 'google_org', 'newletters', 'tofu_emails',
       'affiliates'],
      dtype='object')

Select dates to  match original model

In [ ]:
mmm_data['date_report'] = pd.to_datetime(mmm_data['date_report'] )
mmm_data_dates = mmm_data[ (mmm_data['date_report'] >= '2021-07-27') &  (mmm_data['date_report'] <=  '2022-09-24')]
print(mmm_data_dates["date_report"].min())
print(mmm_data_dates["date_report"].max())

In [39]:
print(mmm_data_dates["date_report"].min())
print(mmm_data_dates["date_report"].max())

2021-07-27 00:00:00
2022-09-24 00:00:00


Training the Model

In [41]:
# set date column
date_col = 'date_report'

# set outcome column
y_col = 'new_clients'

# set marketing variables
channel_cols = ['instagram_org', 
                'paid_meta', 
                'google_search', 
                'google_org', 
                'newletters', 
                'tofu_emails',
                'affiliates']

# set control variables
control_cols = ['sp500']

# split data into features and target
X = mmm_data_dates[[date_col] + channel_cols + control_cols]
y = mmm_data_dates[y_col]

# set test (out-of-sample) length
test_len = 10

# create train and test indices
train_idx = slice(0, len(mmm_data_dates) - test_len)
out_of_time_idx = slice(len(mmm_data_dates) - test_len, len(mmm_data_dates))

In [53]:
mmm_default = MMM(
    adstock=GeometricAdstock(l_max=8),
    saturation=LogisticSaturation(),
    date_column=date_col,
    channel_columns=channel_cols,
    control_columns=control_cols,
)

mmm_default.default_model_config

NameError: name 'MMM' is not defined